## read detection results

In [1]:
import csv

In [156]:
class BBox(object):
    def __init__(self, startX, startY, endX, endY, confidence):
        self.confidence = confidence
        self.startX = startX
        self.startY = startY
        self.endX = endX
        self.endY = endY
        self.width = self.endX-self.startX
        self.height = self.endY-self.startY
        self.match = False
        
    def getWidth(self):
        
        return self.width
    def getHeight(self):
       
        return self.height
    def getArea(self):
        return self.width*self.height
    def getRec(self):
        return np.array([self.startX,self.startY,self.width,self.height])
    def setMatch(self):
        self.match = True
        
        
frame_num = 548


frameBBoxList = [[] for y in range(frame_num)] 


In [158]:

def readFrameBBoxList(frame_num, filename, channel):

    frameBBoxList = [[] for y in range(frame_num)]

    # bbox_count = [[0 for x in range(image_num)] for y in range(channel_num)]
    # ppl_count = [[0 for x in range(image_num)] for y in range(channel_num)]
    
    # area_mat = [[0 for x in range(image_num)] for y in range(channel_num)]
    #


    with open(filename, 'rt') as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile)
        # 以迴圈輸出每一列
        for row in rows:

            channel_idx = int(row[0])
            if (channel_idx == channel):
                img_idx = int(row[1])
                #bbox_count[channel_idx][img_idx] = bbox_count[channel_idx][img_idx] + 1

                confidence = float(row[6])
                startX = int(float(row[2]))
                startY = int(float(row[3]))
                endX = int(float(row[4]))
                endY = int(float(row[5]))
                # already detected one face
                
                frameBBoxList[img_idx].append(BBox(startX, startY, endX, endY, confidence))

    csvfile.close()
    return frameBBoxList

In [212]:
frame_num = 548
channel_num = 7
channel_list = [2, 4, 6]
method = 'faster-rcnn_face'  # results/yolo, results_new/faster-rcnn2, results_new/mask-rcnn
dataset = 'Pathway2_3'
#start_frame = 310
image_num = 550
filename = method + "_" + dataset + ".txt"
channel = 6

frameBBoxList = readFrameBBoxList(frame_num,filename,channel)

for i in range(0,frame_num):
    
    if frameBBoxList[i]:
        for bboxD in frameBBoxList[i]:
            if(bboxD.confidence>0.8):
                print('{} {} {} {}'.format(i,bboxD.getRec(),bboxD.confidence,bboxD.match))
            


107 [639 249  56  69] 0.92105633 False
120 [605 241  36  42] 0.9589241 False
123 [560 222  23  28] 0.9743916 False
138 [455 178  20  22] 0.9147421 False
138 [402 133  24  27] 0.8650491 False
141 [418 167  22  27] 0.9192568 False
142 [416 166  17  20] 0.87442726 False
142 [493 145  21  28] 0.8707321 False
149 [354 123  20  23] 0.866121 False
151 [348 124  18  22] 0.8194072 False
153 [340 117  19  23] 0.83036697 False
154 [343 122  19  21] 0.96787727 False
155 [340 117  22  27] 0.8421982 False
157 [333 119  17  20] 0.9573984 False
162 [305 131  19  22] 0.86012834 False
163 [308 136  17  20] 0.9149918 False
164 [338 296  10  13] 0.9455658 False
165 [319 117  20  26] 0.97482795 False
166 [322 115  10  13] 0.8307207 False
167 [355 134  15  17] 0.9354008 False
167 [381 131  14  15] 0.81041414 False
169 [311 112  17  21] 0.94624823 False
174 [311 113  12  15] 0.9123883 False
174 [349 272   9  12] 0.8607112 False
174 [321 264  10  11] 0.8589348 False
177 [364 121  12  15] 0.8013225 False
178 [

## tracking

In [67]:
import numpy as np
import os
import sys
import time
import argparse
import yaml, json
from PIL import Image

import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim

sys.path.insert(0, '.')
from modules.model import MDNet, BCELoss, set_optimizer
from modules.sample_generator import SampleGenerator
from modules.utils import overlap_ratio
from tracking.data_prov import RegionExtractor
from tracking.bbreg import BBRegressor
from tracking.gen_config import gen_config

opts = yaml.safe_load(open('tracking/options.yaml','r'))


In [68]:
def forward_samples(model, image, samples, out_layer='conv3'):
    model.eval()
    extractor = RegionExtractor(image, samples, opts)
    for i, regions in enumerate(extractor):
        if opts['use_gpu']:
            regions = regions.cuda()
        with torch.no_grad():
            feat = model(regions, out_layer=out_layer)
        if i==0:
            feats = feat.detach().clone()
        else:
            feats = torch.cat((feats, feat.detach().clone()), 0)
    return feats


In [69]:
def train(model, criterion, optimizer, pos_feats, neg_feats, maxiter, in_layer='fc4'):
    model.train()

    batch_pos = opts['batch_pos']
    batch_neg = opts['batch_neg']
    batch_test = opts['batch_test']
    batch_neg_cand = max(opts['batch_neg_cand'], batch_neg)

    pos_idx = np.random.permutation(pos_feats.size(0))
    neg_idx = np.random.permutation(neg_feats.size(0))
    while(len(pos_idx) < batch_pos * maxiter):
        pos_idx = np.concatenate([pos_idx, np.random.permutation(pos_feats.size(0))])
    while(len(neg_idx) < batch_neg_cand * maxiter):
        neg_idx = np.concatenate([neg_idx, np.random.permutation(neg_feats.size(0))])
    pos_pointer = 0
    neg_pointer = 0

    for i in range(maxiter):

        # select pos idx
        pos_next = pos_pointer + batch_pos
        pos_cur_idx = pos_idx[pos_pointer:pos_next]
        pos_cur_idx = pos_feats.new(pos_cur_idx).long()
        pos_pointer = pos_next

        # select neg idx
        neg_next = neg_pointer + batch_neg_cand
        neg_cur_idx = neg_idx[neg_pointer:neg_next]
        neg_cur_idx = neg_feats.new(neg_cur_idx).long()
        neg_pointer = neg_next

        # create batch
        batch_pos_feats = pos_feats[pos_cur_idx]
        batch_neg_feats = neg_feats[neg_cur_idx]

        # hard negative mining
        if batch_neg_cand > batch_neg:
            model.eval()
            for start in range(0, batch_neg_cand, batch_test):
                end = min(start + batch_test, batch_neg_cand)
                with torch.no_grad():
                    score = model(batch_neg_feats[start:end], in_layer=in_layer)
                if start==0:
                    neg_cand_score = score.detach()[:, 1].clone()
                else:
                    neg_cand_score = torch.cat((neg_cand_score, score.detach()[:, 1].clone()), 0)

            _, top_idx = neg_cand_score.topk(batch_neg)
            batch_neg_feats = batch_neg_feats[top_idx]
            model.train()

        # forward
        pos_score = model(batch_pos_feats, in_layer=in_layer)
        neg_score = model(batch_neg_feats, in_layer=in_layer)

        # optimize
        loss = criterion(pos_score, neg_score)
        model.zero_grad()
        loss.backward()
        if 'grad_clip' in opts:
            torch.nn.utils.clip_grad_norm_(model.parameters(), opts['grad_clip'])
        optimizer.step()


In [72]:

def search_track(track_num, frame_idx, init_bbox, previous_num):
    #img_list, init_bbox

    # Init bbox
    
    #result = np.zeros((len(img_list), 4))
    #result_bb = np.zeros((len(img_list), 4))
    #result[0] = target_bbox
    #result_bb[0] = target_bbox

    # superorange params
    track_list = [(-1,-1)]*TRACKLET_NUM
    bbox_list = []
    IOU_count = 0
    # last_bbox = -1
    # next_frame = -1
    frameA_path = os.path.join(FILE_PATH , str(frame_idx) + ".bmp")


    target_bbox = np.array(init_bbox)
    # Init model
    model = MDNet(opts['model_path'])
    if opts['use_gpu']:
        model = model.cuda()

    # Init criterion and optimizer
    criterion = BCELoss()
    model.set_learnable_params(opts['ft_layers'])
    init_optimizer = set_optimizer(model, opts['lr_init'], opts['lr_mult'])
    update_optimizer = set_optimizer(model, opts['lr_update'], opts['lr_mult'])

    tic = time.time()
    # Load first image
    image = Image.open(frameA_path).convert('RGB')

    # Draw pos/neg samples
    pos_examples = SampleGenerator('gaussian', image.size, opts['trans_pos'], opts['scale_pos'])(
        target_bbox, opts['n_pos_init'], opts['overlap_pos_init'])

    neg_examples = np.concatenate([
        SampleGenerator('uniform', image.size, opts['trans_neg_init'], opts['scale_neg_init'])(
            target_bbox, int(opts['n_neg_init'] * 0.5), opts['overlap_neg_init']),
        SampleGenerator('whole', image.size)(
            target_bbox, int(opts['n_neg_init'] * 0.5), opts['overlap_neg_init'])])
    neg_examples = np.random.permutation(neg_examples)

    # Extract pos/neg features
    pos_feats = forward_samples(model, image, pos_examples)
    neg_feats = forward_samples(model, image, neg_examples)

    # Initial training
    train(model, criterion, init_optimizer, pos_feats, neg_feats, opts['maxiter_init'])
    del init_optimizer, neg_feats
    torch.cuda.empty_cache()

    # Train bbox regressor
    bbreg_examples = SampleGenerator('uniform', image.size, opts['trans_bbreg'], opts['scale_bbreg'],
                                     opts['aspect_bbreg'])(
        target_bbox, opts['n_bbreg'], opts['overlap_bbreg'])
    bbreg_feats = forward_samples(model, image, bbreg_examples)
    bbreg = BBRegressor(image.size)
    bbreg.train(bbreg_feats, bbreg_examples, target_bbox)
    del bbreg_feats
    torch.cuda.empty_cache()

    # Init sample generators for update
    sample_generator = SampleGenerator('gaussian', image.size, opts['trans'], opts['scale'])
    pos_generator = SampleGenerator('gaussian', image.size, opts['trans_pos'], opts['scale_pos'])
    neg_generator = SampleGenerator('uniform', image.size, opts['trans_neg'], opts['scale_neg'])

    # Init pos/neg features for update
    neg_examples = neg_generator(target_bbox, opts['n_neg_update'], opts['overlap_neg_init'])
    neg_feats = forward_samples(model, image, neg_examples)
    pos_feats_all = [pos_feats]
    neg_feats_all = [neg_feats]

    spf_total = time.time() - tic




    #fps = len(img_list) / spf_total
    #return result, result_bb, fps

    # Main loop
    for i in range(0,TRACKLET_NUM):  #next 10 frame
        frameB_idx = frame_idx+i+1
        if frameB_idx<frame_num:
            frameB_path = os.path.join(FILE_PATH,str(frameB_idx)+".bmp")

            #------------track by MDNet------------
            # Load image
            image = Image.open(frameB_path).convert('RGB')

            # Estimate target bbox
            samples = sample_generator(target_bbox, opts['n_samples'])
            sample_scores = forward_samples(model, image, samples, out_layer='fc6')

            top_scores, top_idx = sample_scores[:, 1].topk(5)
            top_idx = top_idx.cpu()
            target_score = top_scores.mean()
            target_bbox = samples[top_idx]
            if top_idx.shape[0] > 1:
                target_bbox = target_bbox.mean(axis=0)
            success = target_score > 0

            # Expand search area at failure
            if success:
                sample_generator.set_trans(opts['trans'])
            else:
                sample_generator.expand_trans(opts['trans_limit'])

            # Bbox regression
            if success:
                bbreg_samples = samples[top_idx]
                if top_idx.shape[0] == 1:
                    bbreg_samples = bbreg_samples[None, :]
                bbreg_feats = forward_samples(model, image, bbreg_samples)
                bbreg_samples = bbreg.predict(bbreg_feats, bbreg_samples)
                bbreg_bbox = bbreg_samples.mean(axis=0)
            else:
                bbreg_bbox = target_bbox

            # Save result
            #result[i] = target_bbox
            #result_bb[i] = bbreg_bbox

            #print(target_bbox)
            
            # Data collect
            if success:
                pos_examples = pos_generator(target_bbox, opts['n_pos_update'], opts['overlap_pos_update'])
                pos_feats = forward_samples(model, image, pos_examples)
                pos_feats_all.append(pos_feats)
                if len(pos_feats_all) > opts['n_frames_long']:
                    del pos_feats_all[0]

                neg_examples = neg_generator(target_bbox, opts['n_neg_update'], opts['overlap_neg_update'])
                neg_feats = forward_samples(model, image, neg_examples)
                neg_feats_all.append(neg_feats)
                if len(neg_feats_all) > opts['n_frames_short']:
                    del neg_feats_all[0]

            # Short term update
            if not success:
                nframes = min(opts['n_frames_short'], len(pos_feats_all))
                pos_data = torch.cat(pos_feats_all[-nframes:], 0)
                neg_data = torch.cat(neg_feats_all, 0)
                train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])

            # Long term update
            elif i % opts['long_interval'] == 0:
                pos_data = torch.cat(pos_feats_all, 0)
                neg_data = torch.cat(neg_feats_all, 0)
                train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])

            torch.cuda.empty_cache()


            bboxT = bbreg_bbox
            anyIOU = False
            
            for bbox_id, bboxD in enumerate(frameBBoxList[frameB_idx]):
                
                if bboxD.match == False:
                    IOU_ratio = (bboxD.getRec(),bboxT)
                    if IOU_ratio > 0.3:
                        track_list[i] = (frameB_idx,bbox_id)
                        IOU_count = IOU_count+1
                        bbox_list.append((frameB_idx,bboxD.getRec()))
                        anyIOU = True
                        break
            if not anyIOU:
                bbox_list.append((frameB_idx,bboxT))
                
                
    ##debug   show track_list
    #for idx in range(0,10):
    #    print(track_list[idx])        
    #    print(bbox_list[idx])       
        
        

    if IOU_count >= IOU_count_th: #add track
        for idx in range(0,10):
            bbox_id = track_list[idx][1]
            if bbox_id!=-1:
                #print(frame_idx+idx+1)
                #print(bbox_id)
                frameBBoxList[frame_idx+idx+1][bbox_id].setMatch()

        next_track_frame = -1       
        init_bbox_next = []
        for ii in range(1,11):
            idx = TRACKLET_NUM-ii
            if track_list[idx] !=-1:
                next_track_frame = (-1)*ii #count from back                
                init_bbox_next = bbox_list[idx]
                break
        
        
        #remove overlap range
        start_rm = -1
        while start_rm >=previous_num:
            del GLOBAL_TRACK_LIST[track_num][-1] #rm 1 per move
            #GLOBAL_TRACK_LIST[track_num].remove(-1) 
            start_rm = start_rm -1
        
        if previous_num ==0:
            GLOBAL_TRACK_LIST.append(bbox_list)
        else:
            GLOBAL_TRACK_LIST[track_num].extend(bbox_list)

        next_start_frame = frame_idx+TRACKLET_NUM+next_track_frame
        
        init_bbox_next = bbox_list[TRACKLET_NUM+next_track_frame][1]
        #print(GLOBAL_TRACK_LIST)
        print("( "+str(track_num)+" "+str(next_start_frame)+" "+str(next_track_frame)+")")
        print(init_bbox_next)
        search_track(track_num, next_start_frame, init_bbox_next, next_track_frame)  #next_start_frame = 310  next_track_frame=-3(will be deleted)

In [76]:
# img_list= ["/home/waue0920/Pathway1_1/ORIG/ch6/321.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/322.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/323.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/324.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/325.bmp"]

FILE_PATH = "/home/waue0920/Pathway1_1/ORIG/ch6"
TRACKLET_NUM = 10
IOU_count_th = TRACKLET_NUM/2

GLOBAL_TRACK_LIST = []

for i in range(0,frame_num):    
    if frameBBoxList[i]:
        for bboxD in frameBBoxList[i]:
            if bboxD.match ==False:
                #print(bboxD.getRec())
                print(i)
                
                track_num = len(GLOBAL_TRACK_LIST)
                search_track(track_num, i, bboxD.getRec(), 0)
                
                
                
                





105
114
252
258
263
268
269
341
348
373
519
520
522
523
524
535


In [80]:
TRACK_LIST_CH6 = [[(106, array([  8, 255, 311, 216])), (107, array([  5,  77, 346, 402])), (108, array([ 51, 189, 272, 287])), (109, array([ 96, 213, 225, 259])), (11
, array([112.11865, 155.84021, 257.46597, 252.75815], dtype=float32)), (111, array([135.99837, 149.32623, 255.35393, 243.09915], dtype=float32)), (11
, array([154.78656, 153.57683, 248.74753, 240.22758], dtype=float32)), (113, array([179, 156, 224, 323])), (114, array([207, 147, 199, 331])), (115, 
rray([253.2096 , 141.5473 , 173.97395, 330.37515], dtype=float32)), (116, array([251, 146, 184, 333])), (117, array([255, 140, 188, 331])), (118, arr
y([266, 149, 183, 313])), (119, array([303.39908,  98.69548, 151.58961, 342.66608], dtype=float32)), (120, array([309, 148, 142, 331])), (121, array(
312, 141, 153, 338])), (122, array([324, 139, 147, 313])), (123, array([342.11697,  91.36146, 134.20808, 348.4413 ], dtype=float32)), (124, array([34
.75836 , 140.96884 , 119.466415, 329.32806 ], dtype=float32)), (125, array([330, 126, 139, 343])), (126, array([347, 143, 136, 327])), (127, array([3
6, 131, 129, 324])), (128, array([380, 137, 123, 301])), (129, array([389, 137, 118, 329])), (130, array([388, 127, 118, 323])), (131, array([392, 10
, 116, 322])), (132, array([398, 122, 116, 289])), (133, array([409.35236, 105.93523, 116.6192 , 272.33185], dtype=float32)), (134, array([416, 122, 
03, 278])), (135, array([411, 130, 115, 255])), (136, array([413, 124, 100, 255])), (137, array([414, 121, 101, 258])), (138, array([417, 125,  99, 2
6])), (139, array([417, 123, 102, 225])), (140, array([410, 125, 109, 232])), (141, array([416, 116, 102, 242])), (142, array([410.65912, 117.56864,
94.61869, 215.91382], dtype=float32)), (143, array([409, 121, 106, 223])), (144, array([406, 118, 110, 227])), (145, array([409, 113,  98, 215])), (1
6, array([407, 125, 105, 193])), (147, array([404, 121, 106, 187])), (148, array([407, 117, 103, 188])), (149, array([419, 123,  94, 193])), (150, ar
ay([424, 120,  95, 187])), (151, array([430.84357 , 111.886826,  90.21654 , 184.68579 ], dtype=float32)), (152, array([423, 120, 104, 174])), (153, a
ray([425, 114,  99, 181])), (154, array([424, 116, 104, 174])), (155, array([427, 119, 100, 169])), (156, array([436, 111,  89, 171])), (157, array([
35, 116,  89, 156])), (158, array([432, 114,  88, 156])), (159, array([427, 110,  90, 157])), (160, array([426.8715  , 120.783646,  88.94166 , 145.53
76 ], dtype=float32)), (161, array([424, 117,  91, 156])), (162, array([428, 114,  88, 147])), (163, array([421, 119,  89, 118])), (164, array([421, 
18,  91, 140])), (165, array([413, 106,  96, 150])), (166, array([415, 105,  89, 141])), (167, array([428, 111,  80, 142])), (168, array([425, 112,  
9, 137])), (169, array([428.06293 , 112.778114,  79.76249 , 131.53963 ], dtype=float32)), (170, array([430, 109,  80, 132])), (171, array([434, 107,
81, 137])), (172, array([439, 110,  78, 132])), (173, array([440, 107,  77, 131])), (174, array([442, 108,  77, 124])), (175, array([444, 106,  73, 1
1])), (176, array([445, 104,  72, 133])), (177, array([445, 105,  72, 126])), (178, array([450.3231  , 109.250404,  67.79214 , 118.92354 ], dtype=flo
t32)), (179, array([449, 102,  71, 129])), (180, array([445, 106,  74, 122])), (181, array([448, 106,  68, 125])), (182, array([447, 100,  69, 127]))
(183, array([450, 106,  67, 122])), (184, array([451, 103,  67, 116])), (185, array([454, 102,  62, 118])), (186, array([455, 106,  63, 108])), (187
array([452.1872  ,  97.556076,  64.78285 , 115.29085 ], dtype=float32)), (188, array([456,  99,  62, 117])), (189, array([461, 101,  60, 116])), (19
, array([462,  99,  61, 116])), (191, array([457, 102,  69, 105])), (192, array([461,  97,  64, 114])), (193, array([461, 100,  66, 113])), (194, arr
y([466, 104,  61, 110])), (195, array([471, 100,  62, 111])), (196, array([474.04776, 101.49248,  57.60914, 102.45742], dtype=float32)), (197, array(
473, 100,  63, 108])), (198, array([476,  93,  60, 117])), (199, array([479,  97,  59, 108])), (200, array([487, 102,  54, 106])), (201, array([488, 
01,  52, 104])), (202, array([487, 101,  59, 103])), (203, array([489,  99,  55, 105])), (204, array([491,  92,  58, 115])), (205, array([494.76334 ,
98.991714,  55.758217, 102.128395], dtype=float32)), (206, array([496,  96,  55, 106])), (207, array([496,  96,  58, 100])), (208, array([499,  99,
53, 100])), (209, array([500,  88,  53, 111])), (210, array([499,  87,  56, 110])), (211, array([500,  97,  51, 100])), (212, array([500,  96,  53, 1
1])), (213, array([499,  97,  55,  97])), (214, array([502.25397 ,  93.168594,  47.880783,  98.53683 ], dtype=float32)), (215, array([498,  92,  54,
97])), (216, array([500,  95,  52,  97])), (217, array([504,  94,  51,  97])), (218, array([504,  95,  53,  93])), (219, array([506,  95,  51,  88])),
(220, array([510,  94,  44,  93])), (221, array([506,  94,  51,  91])), (222, array([506,  93,  52,  93])), (223, array([508.80258,  87.41619,  51.08
891,  98.0627 ], dtype=float32)), (224, array([510,  90,  46,  95])), (225, array([511,  97,  46,  86])), (226, array([514,  82,  47, 101])), (227, ar
ray([514,  92,  45,  91])), (228, array([513,  91,  51,  89])), (229, array([519,  89,  43,  93])), (230, array([514,  92,  51,  89])), (231, array([5
17,  94,  50,  82])), (232, array([518.06757,  89.84345,  47.14048,  89.13944], dtype=float32)), (233, array([520,  93,  47,  87])), (234, array([520,
  93,  45,  83])), (235, array([524,  89,  47,  87])), (236, array([523,  94,  48,  81])), (237, array([524,  87,  44,  88])), (238, array([526,  90,
45,  87])), (239, array([526,  94,  46,  80])), (240, array([529,  85,  41,  96])), (241, array([527.79767,  90.1325 ,  44.41456,  83.89948], dtype=f
loat32)), (242, array([528,  87,  45,  84])), (243, array([529,  86,  46,  86])), (244, array([531,  92,  41,  85])), (245, array([531,  82,  44,  90]
)), (246, array([531.0031  ,  91.27719 ,  42.61775 ,  80.765945], dtype=float32)), (247, array([532.0586  ,  90.17743 ,  41.099663,  82.13339 ], dtype
=float32)), (248, array([533,  85,  43,  85])), (249, array([531,  90,  48,  79])), (250, array([531,  93,  50,  77]))], [(374, array([530,  91,  41,
71])), (375, array([532,  92,  38,  67])), (376, array([529,  81,  43,  82])), (377, array([528,  92,  44,  71])), (378, array([532,  87,  37,  76]))
, (379, array([527.22253 ,  81.01827 ,  41.679516,  81.74102 ], dtype=float32)), (380, array([527,  89,  40,  72])), (381, array([527,  95,  43,  66])
), (382, array([524.7393  ,  82.549484,  40.862255,  80.84843 ], dtype=float32)), (383, array([520.08124 ,  89.728645,  42.624367,  75.21013 ], dtype=
float32)), (384, array([520,  64,  47, 103])), (385, array([520,  88,  47,  77])), (386, array([518,  90,  43,  78])), (387, array([518,  96,  42,  68
])), (388, array([514.69934 ,  89.91756 ,  43.981556,  75.046814], dtype=float32)), (389, array([519,  90,  42,  75])), (390, array([518,  91,  39,  7
4])), (391, array([518,  92,  39,  71])), (392, array([514.16113 ,  94.17409 ,  38.41605 ,  73.843376], dtype=float32)), (393, array([517,  92,  38,
68])), (394, array([516,  90,  40,  74])), (395, array([517,  91,  36,  75])), (396, array([512,  96,  43,  73])), (397, array([511,  93,  47,  77])),
(398, array([510,  96,  44,  77])), (399, array([514,  94,  39,  73])), (400, array([514,  91,  40,  79])), (401, array([514.30615,  95.65999,  39.14
743,  74.16192], dtype=float32)), (402, array([514,  95,  39,  75])), (403, array([513,  91,  42,  76])), (404, array([513,  90,  42,  82])), (405, ar
ray([514,  89,  42,  84])), (406, array([514,  88,  49,  83])), (407, array([509,  88,  49,  82])), (408, array([510,  94,  43,  81])), (409, array([5
09,  92,  44,  85])), (410, array([508.97882 ,  99.47112 ,  41.418144,  75.277   ], dtype=float32)), (411, array([510,  96,  43,  81])), (412, array(
509, 100,  45,  78])), (413, array([508, 100,  45,  78])), (414, array([504,  96,  47,  86])), (415, array([503,  99,  49,  82])), (416, array([504,
96,  47,  80])), (417, array([504,  95,  49,  83])), (418, array([505,  97,  47,  86])), (419, array([500.49326,  95.52799,  49.09374,  91.72285], dt
pe=float32)), (420, array([501,  95,  51,  89])), (421, array([501,  97,  47,  84])), (422, array([503,  95,  47,  90])), (423, array([504,  94,  50,
92])), (424, array([503, 100,  50,  86])), (425, array([501,  96,  49,  89])), (426, array([497,  96,  52,  91])), (427, array([497,  97,  51,  92])
, (428, array([500.0284  ,  97.85119 ,  49.269985,  87.98641 ], dtype=float32)), (429, array([500,  98,  51,  89])), (430, array([500, 100,  49,  88]
), (431, array([496,  97,  56,  93])), (432, array([496,  98,  54,  92])), (433, array([496, 100,  54,  92])), (434, array([497,  98,  49,  96])), (4
5, array([494,  98,  57,  96])), (436, array([494, 101,  55,  93])), (437, array([488.81396 ,  97.95134 ,  58.130413,  98.318954], dtype=float32)), (
38, array([491,  99,  55,  97])), (439, array([493, 101,  53, 100])), (440, array([494, 100,  54, 101])), (441, array([491,  98,  58, 103])), (442, a
ray([490, 100,  58, 100])), (443, array([487,  98,  58, 106])), (444, array([486,  99,  56, 102])), (445, array([488, 102,  57, 101])), (446, array([
90.15887 ,  96.26409 ,  55.137012, 104.12948 ], dtype=float32)), (447, array([488, 101,  58, 102])), (448, array([488, 102,  60, 102])), (449, array(
486,  97,  62, 110])), (450, array([487, 102,  61, 101])), (451, array([489, 101,  56, 107])), (452, array([487, 100,  57, 107])), (453, array([484, 
06,  60, 101])), (454, array([480, 100,  63, 112])), (455, array([476.17343 ,  94.23568 ,  60.810497, 117.37805 ], dtype=float32)), (456, array([476,
105,  63, 104])), (457, array([476, 100,  64, 115])), (458, array([477, 100,  64, 116])), (459, array([475, 103,  64, 115])), (460, array([472, 102,
65, 114])), (461, array([465, 100,  71, 116])), (462, array([468, 104,  67, 116])), (463, array([469,  98,  69, 124])), (464, array([470.84668 , 105.
6414 ,  66.89066 , 115.224655], dtype=float32)), (465, array([469, 106,  66, 119])), (466, array([464, 101,  72, 125])), (467, array([462, 104,  73, 
18])), (468, array([462, 105,  70, 120])), (469, array([462,  99,  68, 132])), (470, array([457, 103,  73, 126])), (471, array([452, 104,  73, 132]))
(472, array([444, 104,  76, 129])), (473, array([440.05283, 107.80408,  75.10581, 122.13871], dtype=float32)), (474, array([443, 105,  74, 132])), (
75, array([442, 103,  73, 135])), (476, array([437, 108,  77, 131])), (477, array([434, 108,  75, 138])), (478, array([432, 108,  78, 135])), (479, a
ray([429, 114,  80, 126])), (480, array([430, 107,  80, 141])), (481, array([430, 109,  78, 139])), (482, array([424.63184 , 117.2124  ,  78.107864, 
42.4708  ], dtype=float32)), (483, array([416, 105,  94, 153])), (484, array([413, 106,  96, 145])), (485, array([415, 113,  85, 131])), (486, array(
419, 107,  84, 153])), (487, array([418, 107,  85, 161])), (488, array([419, 114,  83, 163])), (489, array([412, 108,  89, 169])), (490, array([408, 
09,  91, 166])), (491, array([406.76437, 112.72477,  85.38218, 170.96654], dtype=float32)), (492, array([408, 107,  86, 178])), (493, array([404, 113
  91, 163])), (494, array([400, 112,  95, 186])), (495, array([397, 107,  93, 190])), (496, array([389, 113, 102, 180])), (497, array([393, 112,  98,
172])), (498, array([397, 113,  90, 192])), (499, array([388, 118,  98, 181])), (500, array([369.30548, 116.88869, 103.21843, 194.45108], dtype=float
2)), (501, array([366, 117, 111, 187])), (502, array([358, 124, 114, 166])), (503, array([362, 123, 102, 199])), (504, array([358, 119, 103, 196])), 
505, array([357, 127, 101, 200])), (506, array([346, 118, 108, 237])), (507, array([338, 124, 121, 202])), (508, array([334, 128, 116, 224])), (509, 
rray([335.5327  , 114.341446, 109.82501 , 259.91962 ], dtype=float32)), (510, array([344, 120,  99, 253])), (511, array([335, 133, 104, 244])), (512,
array([326, 105, 110, 298])), (513, array([321, 117, 111, 278])), (514, array([312, 130, 117, 290])), (515, array([306.13663, 117.09888, 123.8646 , 2
2.711  ], dtype=float32)), (516, array([312, 129, 105, 294])), (517, array([296, 118, 115, 335])), (518, array([257.50043, 127.07497, 151.57361, 323.
8685], dtype=float32)), (519, array([259, 124, 142, 323])), (520, array([251, 138, 138, 332])), (521, array([229.32373, 129.98125, 144.27982, 338.455
], dtype=float32)), (522, array([232, 107, 131, 369])), (523, array([220, 134, 132, 345])), (524, array([191,  96, 158, 382])), (525, array([171.8133, 151.62027, 166.11711, 335.7473 ], dtype=float32)), (526, array([155.59634, 144.35675, 173.93155, 352.23032], dtype=float32)), (527, array([149.433$ , 153.24356, 177.75465, 327.24625], dtype=float32))], [(536, array([ 22, 149, 252, 330])), (537, array([ 23.973871, 192.4416  , 278.3828  , 289.9669$ ], dtype=float32)), (538, array([ 22.9844 , 179.74057, 279.7392 , 297.88824], dtype=float32)), (539, array([ 21.055899, 197.94269 , 276.19476 , 285.$7578 ], dtype=float32)), (540, array([  7,  63, 288, 416])), (541, array([  0, 208, 294, 268])), (542, array([  4,  54, 254, 425])), (543, array([  2$ 293, 243, 181])), (544, array([  0, 282, 241, 185])), (545, array([  0.     , 233.70337, 217.05135, 270.3214 ], dtype=float32))]]

SyntaxError: invalid syntax (<ipython-input-80-a644a7f131f4>, line 5)

In [210]:
import pickle

bbox_file = 'track_list_faster-rcnn2_Pathway1_1_ch2.pkl'
#reload object from file
file2 = open(bbox_file, 'rb')
bbox_track_list_pickle = pickle.load(file2)
file2.close()

#print dictionary object loaded from file
for idx,track_list in enumerate(bbox_track_list_pickle):
    print("******")
    print(idx)
    for track in track_list:        
        print track
       

******
0
(106, array([  1, 209, 333, 270]))
(107, array([ 13,  74, 326, 405]), 0.9412085)
(108, array([ 51, 172, 262, 298]), 0.92148936)
(109, array([ 37.517235, 164.3206  , 344.6007  , 272.8121  ], dtype=float32), 0)
(110, array([ 69.77383, 153.22871, 334.5662 , 271.83453], dtype=float32), 0)
(111, array([ 65,  42, 355, 437]), 0.8011417)
(112, array([156, 186, 254, 286]), 0.9760223)
(113, array([168, 111, 249, 368]), 0.93068546)
(114, array([198, 166, 199, 307]), 0.9814905)
(115, array([225, 166, 180, 307]), 0.98611116)
(116, array([243, 140, 180, 335]))
(117, array([250, 158, 199, 321]), 0.9971981)
(118, array([254, 174, 212, 285]), 0.99770314)
(119, array([276, 140, 186, 338]), 0.998787)
(120, array([291, 152, 173, 327]), 0.99856156)
(121, array([310, 152, 154, 327]), 0.9992669)
(122, array([322, 154, 148, 320]), 0.99776554)
(123, array([314, 163, 172, 277]), 0.99808323)
(124, array([323, 162, 163, 308]), 0.9988519)
(125, array([338, 151, 137, 307]), 0.99651474)
(126, array([346, 15

## show bbox

In [93]:
import cv2
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

               
output_folder = method + "_" + dataset +"_ch"+str(channel)+"_results"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
    

for idx,track in enumerate(bbox_track_list_pickle):
    color = tableau20[idx]
    for bbox in track:
        file_num = bbox[0]
        filename = os.path.join(FILE_PATH,str(file_num)+".bmp")
        image = cv2.imread(filename)
                                
                                
        startX = bbox[1][0]
        startY = bbox[1][1]
        endX = bbox[1][0]+bbox[1][2]
        endY = bbox[1][1]+bbox[1][3]
        
        
        cv2.rectangle(image, (startX, startY), (endX,endY),color, 2)

        output_name = output_folder + "/track" +str(idx)+"_"+str(file_num)+".jpg"
        print(output_name)
        cv2.imwrite(output_name, image)            
                                
        

faster-rcnn2_Pathway1_1_ch6_results/track0_106.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_107.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_108.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_109.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_110.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_111.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_112.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_113.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_114.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_115.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_116.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_117.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_118.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_119.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_120.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_121.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_122.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_123.jpg
faster-rcnn2_Pathway1_1_ch6_results/track0_124.jpg
faster-rcnn2_Pathway1_1_ch6_res

faster-rcnn2_Pathway1_1_ch6_results/track1_408.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_409.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_410.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_411.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_412.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_413.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_414.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_415.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_416.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_417.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_418.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_419.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_420.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_421.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_422.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_423.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_424.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_425.jpg
faster-rcnn2_Pathway1_1_ch6_results/track1_426.jpg
faster-rcnn2_Pathway1_1_ch6_res

In [152]:
remain = 500
factor=2
n=remain*factor

bbox = np.array([596,70,7,10])
bb = np.array(bb, dtype='float32')

        # (center_x, center_y, w, h)
sample = np.array([bb[0] + bb[2] / 2, bb[1] + bb[3] / 2, bb[2], bb[3]], dtype='float32')
samples = np.tile(sample[None, :], (n ,1))

trans_neg_init = 0.1
scale_neg_init = 1.3
samples[:, :2] += trans_neg_init * np.mean(bb[2:]) * np.clip(0.5 * np.random.randn(n, 2), -1, 1)
samples[:, 2:] *= scale_neg_init ** np.clip(0.5 * np.random.randn(n, 1), -1, 1)


In [153]:
samples_ =samples
overlap_pos_init= [0.7, 1]
overlap_range =overlap_pos_init


In [154]:
# while remain > 0 and factor < 16:
#     samples_ = self._gen_samples(bbox, remain * factor)
print(samples_.shape)
print(samples_)
idx = np.ones(len(samples_), dtype=bool)
if overlap_range is not None:
    r = overlap_ratio(samples_, bbox)

    idx *= (r >= overlap_range[0]) * (r <= overlap_range[1])

print(idx)
samples_ = samples_[idx, :]
print(samples_.shape)
samples_ = samples_[:min(remain, len(samples_))]
print(samples_.shape)

if samples is None:
    samples = samples_
else:
    samples = np.concatenate([samples, samples_])
    
print(samples.shape)

remain = n - len(samples)
factor = factor * 2
print(samples.shape)



(1000, 4)
[[598.9363     75.505196    6.946714    9.923877 ]
 [599.0751     74.764175    7.499234   10.713192 ]
 [599.419      75.56275     5.3846154   7.6923075]
 ...
 [599.2867     75.522026    6.528448    9.326354 ]
 [599.9009     74.15        6.5148354   9.306908 ]
 [599.17145    74.55528     9.1        13.       ]]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False Fa

In [193]:
def overlap_ratio(rect1, rect2):                            
                                                            
    if rect1.ndim == 1:                                     
        rect1 = rect1[None, :]                              
    if rect2.ndim == 1:                                     
        rect2 = rect2[None, :]                              

    left = np.maximum(rect1[:, 0], rect2[:, 0])               
    right = np.minimum(rect1[:, 0] + rect1[:, 2], rect2[:, 0] + rect2[:, 2])                                                
    top = np.maximum(rect1[:, 1], rect2[:, 1])                
    bottom = np.minimum(rect1[:, 1] + rect1[:, 3], rect2[:, 1] + rect2[:, 3])     
    intersect = np.maximum(0, right - left) * np.maximum(0, bottom - top) 
    print(intersect)
    union = rect1[:, 2] * rect1[:, 3] + rect2[:, 2] * rect2[:, 3] - intersect 
    print(union)
    print(intersect / union)
    iou = np.clip(intersect / union, 0, 1)                    
    return iou  

In [204]:
rect447 = np.array([363, 136,  20,  24])

rect4482 = np.array([374, 142,  16,  20])
rect448 = np.array([376, 142,  18,  20])
rect449 = [355.24948 , 138.22949 ,  20.207357,  24.248825]
rect2 = np.array([377.4968  , 145.72443 ,  17.756306,  19.729229])
iou = overlap_ratio(rect448, rect4482)
if iou>0.3:
    print("IOU="+str(iou))

[280]
[400]
[0]


In [184]:
rect2.ndim

1